In [1]:
%load_ext autoreload
%autoreload 2

import re
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import matplotlib
plt.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = 12, 6
matplotlib.rcParams['font.size'] = 18  # Probably OS Dependent

In [3]:
data_files = glob('../public/*.json')


data = []
for file_ in data_files:
    df = pd.read_json(file_)
    false_series = pd.Series(False, index=df.index)
    valid_email = df['emails'].apply(lambda x: len(x) > 0) if 'emails' in df else false_series
    valid_fax = df['faxes'].apply(lambda x: len(x) > 0) if 'faxes' in df else false_series
    valid_email | valid_fax

    data += [{
        'state': re.search('public/(.+).json', file_).group(1).title(),
        'records': df.shape[0],
        'fields': df.shape[1],
        'valid_locale': df['locale'].notnull().mean(),
        'valid_official': df['official'].notnull().mean(),
        'valid_email': valid_email.mean(),
        'valid_faxes': valid_fax.mean(),
        'valid_fax_or_email': (valid_email | valid_fax).mean(),
    }]

pd.DataFrame(data)


,state,records,fields,valid_locale,valid_official,valid_email,valid_faxes,valid_fax_or_email
0,Nebraska,93,8,1.0,1.0,1.000000,1.00000,1.00000
1,Georgia,159,5,1.0,1.0,1.000000,1.00000,1.00000
2,Wisconsin,1853,9,1.0,1.0,0.861846,0.95089,0.95089
3,Maine,504,5,1.0,1.0,0.000000,1.00000,1.00000
4,Maryland,24,7,1.0,1.0,1.000000,0.00000,1.00000
5,Michigan,1535,6,1.0,1.0,1.000000,0.00000,1.00000
6,Florida,67,4,1.0,1.0,1.000000,0.00000,1.00000
7,Minnesota,87,5,1.0,1.0,1.000000,1.00000,1.00000
